In [1]:
!pip install openpyxl --upgrade
!pip install PyPDF2
!pip install pandas
!pip install openpyxl
!pip install xlsxwriter

     |████████████████████████████████| 77 kB 1.4 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61102 sha256=c69971e878086e3800c1c60f928760973d65cc9240a3e4b157d458951eeed866
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2
     |████████████████████████████████| 149 kB 4.3 MB/s 


In [2]:
import pandas as pd
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
from PyPDF2 import PdfFileMerger
from google.colab import files
from matplotlib import rcParams
import altair as alt
import numpy as np

In [3]:
from google.colab import files

# UPLOAD PROGRESS REPORT

progress_report = files.upload()

Saving MyROR Report 2022-03-16.xlsx to MyROR Report 2022-03-16.xlsx


In [4]:
filename = next(iter(progress_report))
df = pd.read_excel(filename, converters={'Site ID': str})


In [5]:
state = input('Enter the abbreviations of the state you want to look up: ')
# Filter by state
df = df[df['Physical Location: State'] == state]

Enter the abbreviations of the state you want to look up: NJ


In [6]:
# Select high level active only sites
high_level_active = ['Active', 'Active - Non-Reporting', 'Slated for deactivation', 'COVID-19 Freeze'] 
df = df[df['Status'].isin(high_level_active)] 

In [7]:
df = df.set_index('Site ID')

In [8]:
df.head()

,Site Type,Site Name,Status,Legacy Satellite ID,Affiliate,Health System,Regions,Categories,Report Name,Report Status,...,COVID-19: Clinic became a testing site,COVID-19: Other impact,COVID-19: clinic converted some 0-5 WCV to telemedicine,COVID-19: Estimated % of WCV converted to telemedicine,COVID-19: Telemedicine EMR coding,COVID-19: How are you incorporating RoR into WCV via telemedicine?,COVID-19: Plan to return to normal,"COVID-19: changes or challenges, anecdotal data that seems important",COVID-19: clinic has literacy rich waiting room,COVID-19: Unable to complete full Report
Site ID,,,,,,,,,,,,,,,,,,,,,
528,Primary,Osborn Family Health Center /,Active,NaN,Reach Out and Read New Jersey,Osborn Family Medical Health Center,NaN,NaN,January 2022 Progress Report,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531,Primary,The Children's Regional Center at Camden / Coo...,Active,NaN,Reach Out and Read New Jersey,The Cooper Health System,NaN,SI: Early Math,January 2022 Progress Report,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531.3,Satellite,The Children's Regional Center at Burlington,Active,1883.0,Reach Out and Read New Jersey,The Cooper Health System,NaN,SI: Early Math,January 2022 Progress Report,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531.4,Satellite,The Children's Regional Center,Active,1976.0,Reach Out and Read New Jersey,The Cooper Health System,NaN,SI: Early Math,January 2022 Progress Report,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531.5,Satellite,Cooper Pediatrics - Washington Township,Active,3158.0,Reach Out and Read New Jersey,The Cooper Health System,NaN,SI: Early Math,January 2022 Progress Report,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df2 = df.filter(regex='Race: ')

In [10]:
df2.head()

,Race: Are you able to provide race demographic data for each of your sites?,Race: White,Race: Black or African American,Race: Hispanic or Latinx,Race: American Indian/Alaska Native,Race: Asian,Race: Native Hawaiian/ Pacific Islander,Race: Unknown / Not Reported,Race: Other - Please Specify,Race: Other 1,Race: Other - Please Specify.1,Race: Other 2
Site ID,,,,,,,,,,,,
528,"No, I would like to use the race demographic d...",0.06,0.21,0.5,NaN,NaN,NaN,NaN,NaN,0.23,NaN,NaN
531,"No, I would like to use the race demographic d...",0.30,0.25,NaN,0.15,0.2,NaN,0.1,NaN,NaN,NaN,NaN
531.3,NaN,0.30,0.25,NaN,0.15,0.2,NaN,0.1,NaN,NaN,NaN,NaN
531.4,NaN,0.30,0.25,NaN,0.15,0.2,NaN,0.1,NaN,NaN,NaN,NaN
531.5,NaN,0.30,0.25,NaN,0.15,0.2,NaN,0.1,NaN,NaN,NaN,NaN


In [11]:
df2 = df2.fillna(0)

In [12]:
#Remove first column
df2 = df2.iloc[: , 1:]

In [13]:
df2.head()

,Race: White,Race: Black or African American,Race: Hispanic or Latinx,Race: American Indian/Alaska Native,Race: Asian,Race: Native Hawaiian/ Pacific Islander,Race: Unknown / Not Reported,Race: Other - Please Specify,Race: Other 1,Race: Other - Please Specify.1,Race: Other 2
Site ID,,,,,,,,,,,
528,0.06,0.21,0.5,0.00,0.0,0.0,0.0,0,0.23,0,0.0
531,0.30,0.25,0.0,0.15,0.2,0.0,0.1,0,0.00,0,0.0
531.3,0.30,0.25,0.0,0.15,0.2,0.0,0.1,0,0.00,0,0.0
531.4,0.30,0.25,0.0,0.15,0.2,0.0,0.1,0,0.00,0,0.0
531.5,0.30,0.25,0.0,0.15,0.2,0.0,0.1,0,0.00,0,0.0


In [14]:
df2.columns = df2.columns.str.replace("Race: ", "")

In [15]:
df2.head()

,White,Black or African American,Hispanic or Latinx,American Indian/Alaska Native,Asian,Native Hawaiian/ Pacific Islander,Unknown / Not Reported,Other - Please Specify,Other 1,Other - Please Specify.1,Other 2
Site ID,,,,,,,,,,,
528,0.06,0.21,0.5,0.00,0.0,0.0,0.0,0,0.23,0,0.0
531,0.30,0.25,0.0,0.15,0.2,0.0,0.1,0,0.00,0,0.0
531.3,0.30,0.25,0.0,0.15,0.2,0.0,0.1,0,0.00,0,0.0
531.4,0.30,0.25,0.0,0.15,0.2,0.0,0.1,0,0.00,0,0.0
531.5,0.30,0.25,0.0,0.15,0.2,0.0,0.1,0,0.00,0,0.0


In [16]:
df2 = df2.drop(columns=['Unknown / Not Reported', 'Other - Please Specify', 'Other 1', 'Other - Please Specify.1', 'Other 2', 'White' ])

In [17]:
df2.head()

,Black or African American,Hispanic or Latinx,American Indian/Alaska Native,Asian,Native Hawaiian/ Pacific Islander
Site ID,,,,,
528,0.21,0.5,0.00,0.0,0.0
531,0.25,0.0,0.15,0.2,0.0
531.3,0.25,0.0,0.15,0.2,0.0
531.4,0.25,0.0,0.15,0.2,0.0
531.5,0.25,0.0,0.15,0.2,0.0


In [18]:
total_BIPOC = df2.sum(axis=1)

In [19]:
print(total_BIPOC)

Site ID
528        0.71
531        0.60
531.3      0.60
531.4      0.60
531.5      0.60
           ... 
18480      0.90
18713      0.45
18738      0.80
18738.2    0.80
19174      0.44
Length: 98, dtype: float64


In [20]:
df2['Total BIPOC'] = total_BIPOC 

In [21]:
df2.head()

,Black or African American,Hispanic or Latinx,American Indian/Alaska Native,Asian,Native Hawaiian/ Pacific Islander,Total BIPOC
Site ID,,,,,,
528,0.21,0.5,0.00,0.0,0.0,0.71
531,0.25,0.0,0.15,0.2,0.0,0.60
531.3,0.25,0.0,0.15,0.2,0.0,0.60
531.4,0.25,0.0,0.15,0.2,0.0,0.60
531.5,0.25,0.0,0.15,0.2,0.0,0.60


In [22]:
majority_BIPOC_list = []

for i, rows in df2.iterrows():
  print(i)
  print(df2.loc[i, 'Total BIPOC'])
  if df2.loc[i, 'Total BIPOC'] > .5:
    majority_BIPOC_list.append(df2.loc[i])

528
0.71
531
0.6000000000000001
531.3
0.6000000000000001
531.4
0.6000000000000001
531.5
0.6000000000000001
531.6
0.6000000000000001
531.7
0.6000000000000001
532
0.9999999999999999
537
0.8800000000000001
537.2
0.8800000000000001
539
0.78
540
0.9
540.2
0.9
541
0.9500000000000001
1470
0.9
1579
0.9000000000000001
1622
0.9500000000000002
1622.2
0.8
1640
0.91
1723
0.91
1723.4
0.91
1723.5
0.91
1723.6
0.91
1823
0.8900000000000001
1959
0.18
1959.3
0.0
1959.4
0.45
2765
0.7
2767
0.6
2780
0.4
2826
0.37
2885
0.64
2885.2
0.64
2885.3
0.64
2885.4
0.64
2885.5
0.64
2885.6
0.64
3653
0.7000000000000001
3656
0.7000000000000001
4153
0.98
4153.2
0.9500000000000001
4271
0.99
4293
0.7
4293.2
0.7
4293.3
0.7
4351
0.9000000000000001
4351.2
0.52
4351.3
0.6200000000000001
4351.4
0.75
4351.5
0.79
4351.6
0.8
4351.7
0.89
4466
0.7000000000000001
4487
0.9
4569
0.66
4597
0.0
4680
0.33999999999999997
5305
0.92
5494
0.5
6075
0.7
6316
0.0
6354
0.99
6376
0.8500000000000001
6417
0.8300000000000001
6446
0.91
6447
0.75000000000

In [23]:
majority_BIPOC_df = pd.DataFrame(majority_BIPOC_list)

In [24]:
majority_BIPOC_df = majority_BIPOC_df.sort_values(by="Total BIPOC", ascending = False)

In [25]:
majority_BIPOC_df.head()

,Black or African American,Hispanic or Latinx,American Indian/Alaska Native,Asian,Native Hawaiian/ Pacific Islander,Total BIPOC
532,0.30,0.60,0.0,0.10,0.0,1.00
6519,0.69,0.30,0.0,0.00,0.0,0.99
8869,0.01,0.97,0.0,0.01,0.0,0.99
6354,0.90,0.08,0.0,0.01,0.0,0.99
4271,0.85,0.14,0.0,0.00,0.0,0.99


In [26]:
majority_BIPOC_filename = 'Majority_BIPOC_Sites_' + state +".xlsx"
writer = pd.ExcelWriter(majority_BIPOC_filename, engine='xlsxwriter')

In [27]:
majority_BIPOC_df.to_excel(writer, sheet_name = "Sites w. BIPOC Majority")

In [28]:
writer.save()
files.download(majority_BIPOC_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
list_race = ['Black or African American', 'Hispanic or Latinx', 'American Indian/Alaska Native', 'Asian', 'Native Hawaiian/ Pacific Islander']

In [51]:
print(list_race)

['Black or African American', 'Hispanic or Latinx', 'American Indian/Alaska Native', 'Asian', 'Native Hawaiian/ Pacific Islander']


In [52]:
source = df2

In [53]:
x_axis_ticks = list(np.linspace(0, 1, num=11))

In [54]:
print(x_axis_ticks)

[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0]


In [64]:
y_axis_ticks = list(np.linspace(0,20,num=21))

In [65]:
print(y_axis_ticks)

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0]


In [72]:
title_name_breakdown = "Histogram of Race & Ethnicity of Families per Site (" +state + " - Majority Breakdown)"

In [74]:
chart = alt.Chart(source).transform_fold(
    list_race,
    as_=['Race', 'Percentage']
).mark_bar(
    opacity=.3,
    binSpacing=1,
    clip=True
).encode(
    x = alt.X('Percentage:Q', bin=alt.Bin(maxbins=50),scale=alt.Scale(domain=(.5, 1)),  axis=alt.Axis(format='%', title='Percentage of Family Demographics per Site',  values=x_axis_ticks)),
    y = alt.Y('count()', stack=None, scale=alt.Scale(domain=(0, 10)), axis=alt.Axis(title='Number of Sites', values=y_axis_ticks)),
    color = alt.Color('Race:N', scale=alt.Scale(scheme='dark2'))

).properties(
    title=title_name_breakdown
)


chart.configure_title(
    fontSize=20,
    font='Tahoma',
    anchor='start',
    color='gray'
)

alt.Chart(...)

In [75]:
total_BIPOC_list = ['Total BIPOC']
title_name_total = "Histogram of Race & Ethnicity of Families per Site (" +state + " - Majority Total)"

In [79]:
chart = alt.Chart(source).transform_fold(
    total_BIPOC_list,
    as_=['Race', 'Percentage']
).mark_bar(
    opacity=.3,
    binSpacing=1,
    clip=True
).encode(
    x = alt.X('Percentage:Q', bin=alt.Bin(maxbins=50),scale=alt.Scale(domain=(.5, 1)),  axis=alt.Axis(format='%', title='Percentage of Family Demographics per Site',  values=x_axis_ticks)),
    y = alt.Y('count()', stack=None, scale=alt.Scale(domain=(0, 15)), axis=alt.Axis(title='Number of Sites', values=y_axis_ticks)),
    color = alt.Color('Race:N', scale=alt.Scale(scheme='dark2'))

).properties(
    title=title_name_total
)


chart.configure_title(
    fontSize=20,
    font='Tahoma',
    anchor='start',
    color='gray'
)

alt.Chart(...)